In [13]:
import numpy as np
import pandas as pd
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [14]:
#what does the data look like

train_df.iloc[:1,:]


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1


In [15]:
# what do disaster tweets looke like

disasters = train_df[train_df["target"]==1]
disasters.iloc[:2,:]

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1


In [16]:
#what do non disaster tweets look like
print ("non_disaster tweet")
non_disasters = train_df[train_df["target"]==0]
non_disasters.iloc[:2,:]

non_disaster tweet


,id,keyword,location,text,target
15,23,NaN,NaN,What's up man?,0
16,24,NaN,NaN,I love fruits,0


In [17]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([
    ('vect', feature_extraction.text.CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])

text_clf.fit(train_df["text"], train_df["target"])

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [24]:
from sklearn.model_selection import GridSearchCV
parameters = {
    'vect__ngram_range': [(1, 1), (1, 2)],
    'tfidf__use_idf': (True, False),
    'clf__alpha': (1e-2, 1e-3),
}
gs_clf = GridSearchCV(text_clf, parameters, cv=5, n_jobs=-1)
gs_clf = gs_clf.fit(train_df["text"], train_df["target"])
gs_clf.best_score_

0.716012084592145

In [19]:
#scores = model_selection.cross_val_score(tfidf, train_vectors, train_df["target"], cv=3, scoring="f1")
#scores
predicted = gs_clf.predict(test_df["text"])

In [20]:
sample_submission = pd.read_csv("sample_submission.csv")
sample_submission["target"] = gs_clf.predict(test_df["text"])
sample_submission.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [21]:
sample_submission.to_csv("submission5-TFIDPipeGridCV.csv", index=False)